In [1]:
#######################################################################################################################
#######################################################################################################################
# 5_c_Calculation of performance criteria with confidence interval: Naive Bayesian models without tf-idf transformation
#######################################################################################################################
#######################################################################################################################

In [2]:
import os # File management
import string # Character chain management
import pandas # Import data into DataFrame
from sklearn.model_selection import train_test_split # Selection abstract for learning corpus  
from sklearn.feature_extraction.text import CountVectorizer # Matrix construction for learning
from sklearn.linear_model import LogisticRegression # Logistic regression
from sklearn import metrics # Confusion matrix
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.feature_extraction.text import TfidfTransformer # For reverse function
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from math import sqrt

In [3]:
# Launch the "import and cleaning abstracts" program
%run "C:\\Users\\....\\Program\\2_Import and cleaning abstracts.ipynb"


In [5]:
# Parameters which may vary: sample size, root, beta for F1
###########################################################

# We vary the sample size, then the root

# Initialization of lists
vrai_neg=[]
faux_neg=[]
vrai_pos=[]
faux_pos=[]
f1=[]
f5=[]

#### We use the Naive Bayesien model without tfidf transformation

# Creation of a list with all the desired sample sizes
echantillon=[0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

for i in echantillon:

    for r in range(1,100):
        # Creation of the training sample (random_state = the seed of the random)
        article_vf_Train,article_vf_Test=train_test_split(articles_vf,train_size=i,random_state=r,stratify=articles_vf['Selection'])

        # Build the analyser with new options: stop_words = 'english' and min_df = 20
        analyseurBis = CountVectorizer(ngram_range=(1,4), stop_words='english',binary=True, min_df=0.04)
        
        # Create the document term matrix
        XTrain_bis = analyseurBis.fit_transform(article_vf_Train['Abstract'])
        
        # The matrix with the terms of the abstracts
        mdtTrainBis = XTrain_bis.toarray()
        
    # Initialize the object
        # Naive Bayesien
        modelNB = MultinomialNB()
       
        # Model execution
        modelNB.fit(mdtTrainBis,article_vf_Train['Selection'])
        
        # Evaluation on the test sample
        mdtTestBis= analyseurBis.transform(article_vf_Test['Abstract'])
        
    # Prediction for abstract test data
        # Naive Bayesien
        predTestBis_NB= modelNB.predict(mdtTestBis)

        # Confusion matrix
        tn, fp, fn, tp = metrics.confusion_matrix(article_vf_Test['Selection'],predTestBis_NB).ravel()
           
        f1.append(fbeta_score(article_vf_Test['Selection'], predTestBis_NB, beta=1))
        f5.append(fbeta_score(article_vf_Test['Selection'], predTestBis_NB, beta=5))
        
        vrai_neg.append(tn)
        faux_pos.append(fp)
        faux_neg.append(fn)
        vrai_pos.append(tp)               


In [6]:
# Transformation into dataframe and excel export
df_vp=pd.DataFrame(np.array(vrai_pos).reshape(9,99))
df_vpt=df_vp.T
df_vn=pd.DataFrame(np.array(vrai_neg).reshape(9,99))
df_vnt=df_vn.T
df_fn=pd.DataFrame(np.array(faux_neg).reshape(9,99))
df_fnt=df_fn.T
df_fp=pd.DataFrame(np.array(faux_pos).reshape(9,99))
df_fpt=df_fp.T

with pd.ExcelWriter(r'C:\\Users\\....\\Result\\matrix_NB.xlsx') as writer:  
    df_vpt.to_excel(writer, sheet_name="True positive")
    df_vnt.to_excel(writer, sheet_name="True negative")
    df_fnt.to_excel(writer, sheet_name="False negative")
    df_fpt.to_excel(writer, sheet_name="False positive")


In [7]:
# Transformation into dataframe and excel export
df1 = pd.DataFrame(np.array(f1).reshape(9,99))

df1t=df1.T
df5 = pd.DataFrame(np.array(f5).reshape(9,99))

df5t=df5.T

with pd.ExcelWriter(r'C:\\Users\\....\\Result\\F1F5_NB.xlsx') as writer:  
    df1t.to_excel(writer, sheet_name="F1")
    df5t.to_excel(writer, sheet_name="F5")
    